In [1]:
import scholarly
import pandas as pd

In [2]:
search_query = scholarly.search_author('Taylor Salo')
author = next(search_query).fill()
publication_titles = [pub.bib['title'] for pub in author.publications]
df = pd.DataFrame(index=publication_titles, columns=['Citations', 'DOI'])
for pub_ in author.publications:
    pub = pub_.fill()
    title = pub.bib['title']
    citedby = [citation.bib['title'] for citation in pub.get_citedby()]
    n_citations = len(citedby)
    df.loc[title, 'Citations'] = n_citations

In [3]:
from Bio import Entrez
from Bio import Medline

# Extract all publications matching term.
Entrez.email = 'tsalo006@fiu.edu'

In [4]:
search = 'Salo[Author] AND {0}'

for title in df.index.tolist():
    s = search.format(title)
    h = Entrez.esearch(db='pubmed', retmax='2', term=s)
    result = Entrez.read(h)
    if int(result['Count']) > 1:
        print('Total number of publications containing {0}: {1}'.format(s, result['Count']))
        df.loc[title, 'DOI'] = None
    elif int(result['Count']) == 0:
        print('Study not found: "{0}"'.format(title))
        df.loc[title, 'DOI'] = None
    else:
        h = Entrez.efetch(db='pubmed', id=result['IdList'], rettype='medline', retmode='text')
        records = Medline.parse(h)
        for record in records:
            print(record.get('TI'))
            doi = [aid for aid in record.get('AID', []) if aid.endswith(' [doi]')]
            if doi:
                doi = doi[0].replace(' [doi]', '')
            else:
                doi = None
            df.loc[title, 'DOI'] = doi

The neural circuitry supporting goal maintenance during cognitive control: a comparison of expectancy AX-CPT and dot probe expectancy paradigms.
Distinct neural correlates for attention lapses in patients with schizophrenia and healthy participants.
Functional network changes and cognitive control in schizophrenia.
Study not found: "Toward a neurobiological basis for understanding learning in University Modeling Instruction physics courses"
Study not found: "Nipype: a flexible, lightweight and extensible neuroimaging data processing framework in Python. 0.14.0"
Study not found: "Nipype: a flexible, lightweight and extensible neuroimaging data processing framework in Python. 0.13.1"


In [5]:
df.to_csv('my_pubs.csv', index_label='Title')